In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_obstructive_pulmonary_disease_(COPD)"
cohort = "GSE162635"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)"
in_cohort_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE162635"

# Output paths
out_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/GSE162635.csv"
out_gene_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE162635.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/clinical_data/GSE162635.csv"
json_path = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
from typing import Optional, Callable, Dict, Any
import json
import numpy as np

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from COPD patients
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (COPD status):
# The key 'gold.1' in sample characteristics dictionary indicates GOLD staging for COPD
# GOLD staging is a classification of COPD severity (0 is chronic bronchitis without obstruction,
# I-IV are COPD stages, and healthy indicates control)
trait_row = 2  # corresponds to 'gold.1' values

# For age:
# Age information is not available in the sample characteristics dictionary
age_row = None

# For gender:
# Gender information is not available in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion

# Convert trait (COPD) function:
def convert_trait(value: str) -> Optional[int]:
    """
    Convert GOLD staging to binary COPD status
    0 = No COPD (healthy controls)
    1 = COPD (including chronic bronchitis without obstruction and GOLD stages I-IV)
    """
    if pd.isna(value):
        return None
    
    # Extract value after colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary COPD status
    if value == "healthy":
        return 0
    elif value in ["O", "I", "II", "III", "IV"]:  # All GOLD stages considered COPD
        return 1
    else:
        return None

# No conversion functions needed for age and gender as they are not available
convert_age = None
convert_gender = None

# 3. Save Metadata
# Initial filtering on usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the clinical data
    # The data is expected to be in a specific format for geo_select_clinical_features
    # We need to construct a DataFrame where rows are features (indexed by numbers)
    # and columns are samples
    
    # Sample characteristics dictionary from previous output
    sample_chars_dict = {
        0: ['tissue: transbronchial biopsy'], 
        1: ['visit: 1', 'visit: 2', 'visit: 3'], 
        2: ['gold.1: O', 'gold.1: II', 'gold.1: III', 'gold.1: I', 'gold.1: healthy', 'gold.1: IV'], 
        3: ['gold.2: O', 'gold.2: II', 'gold.2: III', 'gold.2: I', 'phl.1: Productive', 'gold.2: healthy', 'gold.2: IV'], 
        4: ['gold.3: O', 'gold.3: II', 'gold.3: III', 'gold.3: I', 'gold.3: IV', 'phl.1: Severe', 'phl.1: No_Dry', 'patid: pat019', 'gold.3: healthy', 'phl.1: Productive', 'patid: pat055'], 
        5: ['phl.1: Severe', 'phl.1: Productive', 'phl.1: No_Dry', 'phl.2: Productive', 'phl.2: No_Dry', np.nan, 'phl.1: Healthy', 'phl.2: Severe'], 
        6: ['phl.2: Severe', 'phl.2: Productive', 'phl.2: No_Dry', 'patid: pat013', 'patid: pat014', np.nan, 'phl.2: Healthy', 'patid: pat031', 'patid: pat039', 'patid: pat042', 'patid: pat052', 'patid: pat053', 'patid: pat060', 'phl.3: No_Dry'], 
        7: ['phl.3: Severe', 'phl.3: No_Dry', 'phl.3: Productive', 'patid: pat010', np.nan, 'phl.3: Healthy', 'patid: pat071'], 
        8: ['patid: pat001', 'patid: pat002', 'patid: pat003', 'patid: pat004', 'patid: pat005', 'patid: pat006', 'patid: pat007', 'patid: pat008', 'patid: pat009', np.nan, 'patid: pat011', 'patid: pat012', 'patid: pat015', 'patid: pat016', 'patid: pat017', 'patid: pat018', 'patid: pat020', 'patid: pat021', 'patid: pat022', 'patid: pat023', 'patid: pat024', 'patid: pat025', 'patid: pat026', 'patid: pat027', 'patid: pat028', 'patid: pat029', 'patid: pat030', 'patid: pat032', 'patid: pat033', 'patid: pat034']
    }
    
    # Find the maximum number of samples
    max_samples = max(len(values) for values in sample_chars_dict.values())
    
    # Create sample names
    sample_names = [f"GSM{i+1}" for i in range(max_samples)]
    
    # Create a DataFrame with the right structure
    clinical_data = pd.DataFrame(index=list(range(max(sample_chars_dict.keys()) + 1)),
                                columns=sample_names)
    
    # Fill in the data
    for row, values in sample_chars_dict.items():
        for i, value in enumerate(values):
            if i < len(sample_names):  # Make sure we don't go out of bounds
                clinical_data.iloc[row, i] = value
    
    # Extract clinical features
    selected_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(selected_features)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the extracted features to CSV
    selected_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# Review the gene identifiers
# The identifiers (121_at, 1316_at, etc.) appear to be Affymetrix probe IDs from an Affymetrix microarray platform
# These are not human gene symbols but probe identifiers that need to be mapped to gene symbols
# The "_at" suffix is characteristic of Affymetrix probe IDs

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the columns for gene identifier and gene symbol in the annotation
# From the preview, we can see:
# - 'ID' column contains the probe IDs (like "1007_s_at") which match the gene expression data row indices
# - 'Gene Symbol' column contains the gene symbols we want to map to

# 2. Get a gene mapping dataframe
# Use get_gene_mapping to extract the gene identifier and gene symbol columns
# Notice that "Gene Symbol" includes multiple genes per probe (e.g., "DDR1 /// MIR4640")
prob_col = 'ID'
gene_col = 'Gene Symbol'
mapping_data = get_gene_mapping(gene_annotation, prob_col, gene_col)

print(f"Gene mapping shape: {mapping_data.shape}")
print("Gene mapping sample:")
print(mapping_data.head())

# 3. Apply the gene mapping to the gene expression data
# This function will handle probes mapped to multiple genes
gene_data = apply_gene_mapping(gene_data, mapping_data)
print(f"Gene data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])

# Save the mapped gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Try to load the clinical data if it exists
try:
    clinical_features = pd.read_csv(out_clinical_data_file, index_col=0)
    print(f"Loaded clinical features shape: {clinical_features.shape}")
    has_clinical_data = True
except FileNotFoundError:
    print(f"Clinical data file not found at {out_clinical_data_file}")
    has_clinical_data = False
except pd.errors.EmptyDataError:
    print(f"Clinical data file is empty at {out_clinical_data_file}")
    has_clinical_data = False

# Link clinical and genetic data if clinical data exists
if has_clinical_data:
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    
    # Check if trait name is in the dataframe index
    if trait in linked_data.index:
        # 3. Handle missing values
        linked_data_clean = handle_missing_values(linked_data, trait)
        print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")
        
        # 4. Determine whether the trait and demographic features are severely biased
        is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)
        trait_available = True
    else:
        print(f"Trait '{trait}' not found in linked data. Using first row as placeholder.")
        # Using first row as placeholder for handle_missing_values function
        first_row = linked_data.index[0]
        linked_data_clean = handle_missing_values(linked_data, first_row)
        print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")
        is_biased = True  # Mark as biased since no trait information for COPD
        trait_available = False
else:
    # If no clinical features, just use gene expression data
    print("No clinical features available, proceeding with gene expression data only")
    linked_data = normalized_gene_data.T  # Transpose to match expected format
    linked_data_clean = linked_data
    is_biased = True  # Mark as biased since no trait information
    trait_available = False

# 5. Conduct final quality validation
note = "Dataset contains gene expression data from transbronchial biopsies from COPD patients at different GOLD stages."
if not trait_available:
    note += " No usable COPD trait information for associative analysis."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=trait_available,
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Use the normalized data for the rest of the analysis
gene_data = normalized_gene_data

# 2. Since we found out in Step 2 that trait_row is not None, clinical data should be available
try:
    clinical_features = pd.read_csv(out_clinical_data_file, index_col=0)
    print(f"Loaded clinical features shape: {clinical_features.shape}")
    has_clinical_data = True
except (FileNotFoundError, pd.errors.EmptyDataError):
    print(f"Clinical data file not found or empty at {out_clinical_data_file}")
    has_clinical_data = False

# Create a new DataFrame with a dummy trait column for the purpose of data handling
if has_clinical_data:
    # Check if the rows in clinical_features are actually traits
    if trait in clinical_features.index:
        # Proper clinical data with trait information
        linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # Handle missing values using the trait as the key column
        linked_data_clean = handle_missing_values(linked_data, trait)
        print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")
        
        # Determine whether trait and demographic features are biased
        is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)
        trait_available = True
    else:
        # The clinical data file exists but doesn't have the trait information
        print(f"Trait '{trait}' not found in clinical data.")
        trait_available = False
        has_clinical_data = False  # Treat as if no clinical data
else:
    # No clinical data available
    trait_available = False

# If no proper clinical data with trait, use gene expression data only
if not has_clinical_data:
    print("No usable clinical features available, proceeding with gene expression data only")
    linked_data = gene_data.T  # Transpose to match expected format
    linked_data_clean = linked_data
    is_biased = True  # Mark as biased since no trait information

# 5. Conduct final quality validation
note = "Dataset contains gene expression data from transbronchial biopsies from COPD patients at different GOLD stages. "
if not trait_available:
    note += "No usable COPD trait information for associative analysis."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=trait_available,
    is_biased=is_biased if trait_available else True,
    df=linked_data_clean,
    note=note
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")